# Data validation

## Setup

In [1]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────────────────────────────── tidyverse 1.3.0 ──

✓ ggplot2 3.3.2     ✓ purrr   0.3.4
✓ tibble  3.0.3     ✓ dplyr   1.0.0
✓ tidyr   1.1.0     ✓ stringr 1.4.0
✓ readr   1.3.1     ✓ forcats 0.5.0

── Conflicts ────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()



In [2]:
na_summary <- arrow::read_feather("data/na_summary.feather")

head(na_summary)

rowtype,boardname,ned,directorname,rolename,rolestatus,gender,nationality,boardid,clientcompanyid,⋯,nationalitymix,numberdirectors,stdevtimebrd,stdevtimeinco,stdevtotnolstdbrd,stdevtotcurrnolstdbrd,stdevnoquals,stdevage,networksize,companyid
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<???>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Board Member,EQUITY ONE INC (De-listed 03/2017),Yes,David Fischel,Independent Director,David Fischel joined this role on 04 Jan 2011,M,British,10925,NA,⋯,0.4,9,7.1,7.1,3.1,1.3,1.5,8.5,6092,10925
Board Member,EQUITY ONE INC (De-listed 03/2017),Yes,David Fischel,Independent Director,David Fischel joined this role on 04 Jan 2011,M,British,10925,NA,⋯,0.4,9,5.8,5.8,3.2,1.2,1.2,7.7,6092,10925
Board Member,EQUITY ONE INC (De-listed 03/2017),Yes,David Fischel,Independent Director,David Fischel joined this role on 04 Jan 2011,M,British,10925,NA,⋯,0.4,10,5.3,5.3,3.0,1.8,1.1,6.9,6092,10925
Board Member,EQUITY ONE INC (De-listed 03/2017),Yes,David Fischel,Independent Director,David Fischel joined this role on 04 Jan 2011,M,British,10925,NA,⋯,0.4,9,7.1,7.1,3.1,1.4,1.5,8.5,6092,10925
Board Member,EQUITY ONE INC (De-listed 03/2017),Yes,David Fischel,Independent Director,David Fischel joined this role on 04 Jan 2011,M,British,10925,NA,⋯,0.5,10,5.6,5.6,3.1,1.5,1.1,7.3,6092,10925
Board Member,NATIONAL MEDICAL HEALTH CARD SYSTEMS INC (De-listed 04/2008),Yes,David Shaw,Director - SD,David Shaw joined this role on 08 Dec 2004,M,American,21616,NA,⋯,0.0,10,2.2,2.1,1.4,0.7,0.8,9.4,4321,21616


## Check for duplicates

### Annual report inconsistencies

If there are no inconsistent, the number of unique entries should be the same, regardless of whether or not we include only company and year, or include variables of interest, too.

In [3]:
na_summary %>%
    select(companyid, annualreportdate) %>%
    n_distinct()

na_summary %>%
    select(companyid, annualreportdate, numberdirectors, nationalitymix, genderratio, companyid) %>%
    n_distinct()

[1] 130393

[1] 130393

Looks correct.

In [4]:
na_summary %>%
    select(companyid, annualreportdate) %>%
    n_distinct()

na_summary %>%
    select(companyid, annualreportdate, isin) %>%
    n_distinct()

[1] 130393

[1] 139359

There are 9,040 inconsistencies with the ISIN code.

### Multiple annual reports in a year

If there are multple annual reports in any year, the number of entries should be reduced when we extract only the year from the annual report date column.

In [5]:
na_summary %>%
    select(companyid, annualreportdate) %>%
    n_distinct()

[1] 130393

In [6]:
na_summary %>%
    mutate(year = lubridate::year(annualreportdate)) %>%
    select(companyid, year) %>%
    n_distinct()

[1] 129761

There are 632 instances of two reports being filled in one year.